<a href="https://colab.research.google.com/github/ccc-frankfurt/Practical_ML_WS19/blob/master/week10/VAE_MLPR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Variational Autoencoders
Here, we extend our previous MLP and CNN PyTorch Fashion-MNIST autoencoder example to variational autoencoders and thus generative models. 

Before starting the notebook you should make sure that your runtime uses GPU acceleration. You can find the corresponding option under *runtime* and then *change runtime type*.

In [0]:
import numpy as np
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import matplotlib.pyplot as plt
%matplotlib inline
print(torch.__version__)

1.3.1


### Dataset class in PyTorch
Our dataset loader essentially remains the same as before. We do not remove the labels to avoid changes to the code, but they are no longer necessary and we will not use them.

In [0]:
import os
import struct
import gzip
import errno
import torch.utils.data
import torchvision.datasets as datasets


class FashionMNIST:
    """
    Fashion MNIST dataset featuring gray-scale 28x28 images of
    fashion items belonging to ten different classes.
    Dataloader adapted from MNIST.
    We do not define __getitem__ and __len__ in this class
    as we are using torch.utils.data.TensorDataSet which
    already implements these methods.

    Parameters:
        args (dict): Dictionary of (command line) arguments.
            Needs to contain batch_size (int) and workers(int).
        is_gpu (bool): True if CUDA is enabled.
            Sets value of pin_memory in DataLoader.

    Attributes:
        trainset (torch.utils.data.TensorDataset): Training set wrapper.
        valset (torch.utils.data.TensorDataset): Validation set wrapper.
        train_loader (torch.utils.data.DataLoader): Training set loader with shuffling.
        val_loader (torch.utils.data.DataLoader): Validation set loader.
    """

    def __init__(self, is_gpu, batch_size, workers):
        self.path = os.path.expanduser('datasets/FashionMNIST')
        self.__download()

        self.trainset, self.valset = self.get_dataset()

        self.train_loader, self.val_loader = self.get_dataset_loader(batch_size, workers, is_gpu)

        self.val_loader.dataset.class_to_idx = {'T-shirt/top': 0,
                                                'Trouser': 1,
                                                'Pullover': 2,
                                                'Dress': 3,
                                                'Coat': 4,
                                                'Sandal': 5,
                                                'Shirt': 6,
                                                'Sneaker': 7,
                                                'Bag': 8,
                                                'Ankle boot': 9}

    def __check_exists(self):
        """
        Checks if dataset has already been downloaded

        Returns:
             bool: True if downloaded dataset has been found
        """

        return os.path.exists(os.path.join(self.path, 'train-images-idx3-ubyte.gz')) and \
               os.path.exists(os.path.join(self.path, 'train-labels-idx1-ubyte.gz')) and \
               os.path.exists(os.path.join(self.path, 't10k-images-idx3-ubyte.gz')) and \
               os.path.exists(os.path.join(self.path, 't10k-labels-idx1-ubyte.gz'))

    def __download(self):
        """
        Downloads the Fashion-MNIST dataset from the web if dataset
        hasn't already been downloaded.
        """

        from six.moves import urllib

        if self.__check_exists():
            return

        print("Downloading FashionMNIST dataset")
        urls = [
            'https://cdn.rawgit.com/zalandoresearch/fashion-mnist/ed8e4f3b/data/fashion/train-images-idx3-ubyte.gz',
            'https://cdn.rawgit.com/zalandoresearch/fashion-mnist/ed8e4f3b/data/fashion/train-labels-idx1-ubyte.gz',
            'https://cdn.rawgit.com/zalandoresearch/fashion-mnist/ed8e4f3b/data/fashion/t10k-images-idx3-ubyte.gz',
            'https://cdn.rawgit.com/zalandoresearch/fashion-mnist/ed8e4f3b/data/fashion/t10k-labels-idx1-ubyte.gz',
        ]

        # download files
        try:
            os.makedirs(self.path)
        except OSError as e:
            if e.errno == errno.EEXIST:
                pass
            else:
                raise

        for url in urls:
            print('Downloading ' + url)
            data = urllib.request.urlopen(url)
            filename = url.rpartition('/')[2]
            file_path = os.path.join(self.path, filename)
            with open(file_path, 'wb') as f:
                f.write(data.read())

        print('Done!')

    def __get_fashion_mnist(self, path, kind='train'):
        """
        Load Fashion-MNIST data

        Parameters:
            path (str): Base directory path containing .gz files for
                the Fashion-MNIST dataset
            kind (str): Accepted types are 'train' and 't10k' for
                training and validation set stored in .gz files

        Returns:
            numpy.array: images, labels
        """

        labels_path = os.path.join(path,
                                   '%s-labels-idx1-ubyte.gz'
                                   % kind)
        images_path = os.path.join(path,
                                   '%s-images-idx3-ubyte.gz'
                                   % kind)

        with gzip.open(labels_path, 'rb') as lbpath:
            struct.unpack('>II', lbpath.read(8))
            labels = np.frombuffer(lbpath.read(), dtype=np.uint8)

        with gzip.open(images_path, 'rb') as imgpath:
            struct.unpack(">IIII", imgpath.read(16))
            images = np.frombuffer(imgpath.read(), dtype=np.uint8).reshape(len(labels), 784)

        return images, labels

    def get_dataset(self):
        """
        Loads and wraps training and validation datasets

        Returns:
             torch.utils.data.TensorDataset: trainset, valset
        """

        x_train, y_train = self.__get_fashion_mnist(self.path, kind='train')
        x_val, y_val = self.__get_fashion_mnist(self.path, kind='t10k')

        # This is new with respect to our previous data loader
        # convert to torch tensors in range [0, 1]
        x_train = torch.from_numpy(x_train).float() / 255
        y_train = torch.from_numpy(y_train).long()
        x_val = torch.from_numpy(x_val).float() / 255
        y_val = torch.from_numpy(y_val).long()

        # resize flattened array of images for input to a CNN
        # we use the in-place variant of the resize function here
        x_train.resize_(x_train.size(0), 1, 28, 28)
        x_val.resize_(x_val.size(0), 1, 28, 28)

        # TensorDataset wrapper
        trainset = torch.utils.data.TensorDataset(x_train, y_train)
        valset = torch.utils.data.TensorDataset(x_val, y_val)

        return trainset, valset

    def get_dataset_loader(self, batch_size, workers, is_gpu):
        """
        Defines the dataset loader for wrapped dataset

        Parameters:
            batch_size (int): Defines the batch size in data loader
            workers (int): Number of parallel threads to be used by data loader
            is_gpu (bool): True if CUDA is enabled so pin_memory is set to True

        Returns:
             torch.utils.data.TensorDataset: trainset, valset
        """

        # multi-threaded data loaders
        train_loader = torch.utils.data.DataLoader(self.trainset, batch_size=batch_size, shuffle=True,
                                                   num_workers=workers, pin_memory=is_gpu, sampler=None)
        test_loader = torch.utils.data.DataLoader(self.valset, batch_size=batch_size, shuffle=True,
                                                  num_workers=workers, pin_memory=is_gpu, sampler=None)

        return train_loader, test_loader


Let's load the data and set the device to use. 

In [0]:
# set a boolean flag that indicates whether a cuda capable GPU is available 
# we will need this for transferring our tensors to the device and 
# for persistent memory in the data loader
is_gpu = torch.cuda.is_available()
print("GPU is available:", is_gpu)
print("If you are receiving False, try setting your runtime to GPU")

# set the device to cuda if a GPU is available
device = torch.device("cuda" if is_gpu else "cpu")

# in contrast to our MLP from scratch notebook, we need to set the batch size already now
# this is because our data loader now requires it.
batch_size = 128
# we also set the amount of workers, i.e. parallel threads to use in our data loader
workers = 4

# We can now instantiate our dataset class 
dataset = FashionMNIST(is_gpu, batch_size, workers)

GPU is available: True
If you are receiving False, try setting your runtime to GPU


# Variational Autoencoder
We have seen how we can encode data into a latent vector and decode back to the original image for training with reconstruction loss.  

In the lecture we have encountered another Bayesian variant, called the variational autoencoder that provides a different perspective with a powerful generative model: https://arxiv.org/abs/1312.6114 . 
We will now minimize the variational lower bound on the evidence (ELBO). To approximate the true posterior to the generative model p(x, z) we will use our neural network encoder to calculate q(x|z) (the approximate posterior). This is sometimes also referred to as the recognition model.

The probabilistic decoder p(x|z) in turn computes the probability density of an input x under the generative model given a sample z from the approximate posterior q(z|x). Like in the autoencoder the parameters of encoder and decoder are optimized jointly. 

We can conduct this optimization using the reparameterization trick, where we express the random variable z through a deterministic variable as seen in the lecture.

In practical terms, in contrast to the regular autoencoder, we thus now optimize for both a reconstruction loss and KL divergence between our prior and approximate posterior. 

Following the lecture, we will use a Gaussian prior and implement the corresponding reparameterization.

A good in-depth introduction to variational auto-encoders for further reading can be found here: https://arxiv.org/abs/1906.02691

## Building our CNN model
Last week we have implemented our PyTorch models with nn.Sequential() containers in order to easily access the model.encoder to train a classifier on top. Prior to that we have seen how we can define our model's layers in the class' constructor and use a forward function to define the execution. This week we will encounter another third variant, where we define multiple custom functions called encode, decode and reparameterize which we can then individually access and define a joint forward function. 

In [0]:
class VAE_CNN(nn.Module):
    def __init__(self, latent_dim):
        super(VAE_CNN, self).__init__()
        
        self.latent_dim = latent_dim 
        
        self.conv1 = nn.Conv2d(1, 64, 5) # input features, output features, kernel size
        self.mp1 = nn.MaxPool2d(2, 2) # kernel size, stride
        
        self.conv2 = nn.Conv2d(64, 128, 5) # input features, output features, kernel size
        self.mp2 = nn.MaxPool2d(2, 2) # kernel size, stride
        
        # tip: 4x4 is the remaining spatial resolution here
        self.latent_mu = ...
        self.latent_sigma = ...
        
        # decoder layers
        # implement the decoder layers 
        ...
        self.Upsample = nn.Upsample(scale_factor=2, mode='nearest')
        
    def encode(self, x):
        # define the probabilistic encoding
        ...
        return mu, sigma
        
    def reparameterize(self, mu, std):
        # define the reparameterization: z = eps * sigma + mu
        # where eps ~ N(0, 1)
        ...
        return z
    
    def decode(self, z):
        # define the probabilistic decoding
        ...
        return x
    
    def forward(self, x):
        # define the end-to-end forward function
        ...

        # make sure to return output, was well as latent mu and sigma to 
        # calculate the reconstruction and KLD loss terms
        return x, mu, sigma

#### We will now also have to define our loss function
In addition to the reconstruction loss, we need to implement the KL divergence between the approximate posterior, thus our latent mu and sigma and our unit Gaussian prior. 

We again add a convenience class to log our losses

In [0]:
class AverageMeter(object):
    """
    Computes and stores the average and current value
    """
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

and the extended loss function

In [0]:
def VAE_loss_function(output, target, mu, std):
    recon_loss_func = nn.BCEWithLogitsLoss(reduction='sum')

    # compute reconstruction loss - normalize by batch size
    recon_loss = ...
    
    # numerical value for stability of log computation
    eps = 1e-8
    # compute Kullback Leibler Divergence - normalize by batch size
    kld = ...
    
    return recon_loss, kld

We will also need to modify our training and validation functions to include the modified loss and let the model return our latent mu and sigma vectors

In [0]:
def train(train_loader, model, criterion, optimizer, device):
    """
    Trains/updates the model for one epoch on the training dataset.

    Parameters:
        train_loader (torch.utils.data.DataLoader): The trainset dataloader
        model (torch.nn.module): Model to be trained
        criterion (torch.nn.criterion): Loss function
        optimizer (torch.optim.optimizer): optimizer instance like SGD or Adam
        device (string): cuda or cpu
    """

    # create an instance of the average meter to track losses
    losses = AverageMeter()
    recon_losses = AverageMeter()
    kl_losses = AverageMeter()

    # switch to train mode
    model.train()

    # iterate through the dataset loader
    # we can now discard the labels returned by our old data loader as we no longer need them
    for i, (inp, _) in enumerate(train_loader):
        # transfer inputs and targets to the GPU (if it is available)
        inp = inp.to(device)
        target = inp
        
        # you can make your autoencoder a denoising autoencoder by 
        # adding noise to the input, but not to the target!
        # To test the denoising autoencoder, uncomment the line below, 
        # where we add a small Gaussian noise to the original images
        #inp = inp + torch.randn(inp.size()).to(device) * 0.1
        
        # compute output, i.e. the model forward. 
        # In contrast to our autoencoder this now also returns the latent mu
        # and sigma that we need to calculate the KL divergence
        ... = model(...)
        
        # calculate the loss
        recon_loss, kl_loss = criterion(...)
        loss = ...

        # record loss
        losses.update(loss.item(), inp.size(0))
        recon_losses.update(recon_loss.item(), inp.size(0))
        kl_losses.update(kl_loss.item(), inp.size(0))

        # compute gradient and do the SGD step
        # we reset the optimizer with zero_grad to "flush" former gradients
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # print the loss every 100 mini-batches
        if i % 100 == 0:
            print('Loss {loss.val:.4f} ({loss.avg:.4f}) \t'
                  'Reconstruction {recon.val:.4f} ({recon.avg:.4f}) \t'
                  'KLD {kld.val:.4f} ({kld.avg:.4f})'
                  .format(loss=losses, recon=recon_losses, kld=kl_losses))

In [0]:
def validate(val_loader, model, criterion, device):
    """
    Trains/updates the model for one epoch on the training dataset.

    Parameters:
        val_loader (torch.utils.data.DataLoader): The valset dataloader
        model (torch.nn.module): Model to be trained
        criterion (torch.nn.criterion): Loss function
        device (string): cuda or cpu
    """

    # create an instance of the average meter to track losses
    losses = AverageMeter()
    recon_losses = AverageMeter()
    kl_losses = AverageMeter()

    # switch to train mode
    model.eval()

    # iterate through the dataset loader
    # we can now discard the labels returned by our old data loader as we no longer need them
    for i, (inp, _) in enumerate(val_loader):
        # transfer inputs and targets to the GPU (if it is available)
        inp = inp.to(device)
        target = inp
        
        # compute output, i.e. the model forward. 
        # In contrast to our autoencoder this now also returns the latent mu
        # and sigma that we need to calculate the KL divergence
        ... = model(...)
        
        # calculate the loss
        recon_loss, kl_loss = criterion(...)
        loss = ...

        # record loss
        losses.update(loss.item(), inp.size(0))
        recon_losses.update(recon_loss.item(), inp.size(0))
        kl_losses.update(kl_loss.item(), inp.size(0))
        
        # print the loss every 100 mini-batches
        if i % 100 == 0:
            print('Loss {loss.val:.4f} ({loss.avg:.4f}) \t'
                  'Reconstruction {recon.val:.4f} ({recon.avg:.4f}) \t'
                  'KLD {kld.val:.4f} ({kld.avg:.4f})'
                  .format(loss=losses, recon=recon_losses, kld=kl_losses))

In addition to training and validation, we can now also write a generation function. Once the model is training and minimizing the divergence between our approximate posterior distribution and our Gaussian prior, we can start sampling from the prior and directly generate images from the corresponding latent vector z. 

In [0]:
def generate(model, device, num_images=100):
    with torch.no_grad():
        # sample from the Gaussian prior p(z)
        z = ...

        # decode the samples p(x|z)
        # add a sigmoid function at the end of the decoder to constrain 
        # the generated image to 0-1 range (which is otherwise not guaranteed)
        generated_images = ...

        # visualize
        imgs = torchvision.utils.make_grid(generated_images.cpu(),
                                           nrow=int(math.sqrt(generated_images.size(0))),
                                           padding=5)
        npimgs = imgs.numpy()
        # when using matplotlib the color channels are expected to be in the third 
        # instead of the first dimension -> tranpose
        plt.imshow(np.transpose(npimgs, (1,2,0)))
        plt.show()

### Constructing and running the CNN based VAE
Let's create an instance of our VAE model and optimize it. After each epoch we will sample from the prior and generate some example images and took a look at their visual quality.

In [0]:
# create VAE model instance
latent_dim = 50 # like in the autoencoder, this is a hyper-parameter
# a good starting point for the above model is a latent dimension of 50.
model = VAE_CNN(latent_dim).to(device)
print(model)

# again, define loss function and optimizer. This time using our custom loss.
criterion = VAE_loss_function

optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

# optimize and visualize generations from generative model at each epoch
total_epochs = 20
for epoch in range(total_epochs):
    print("EPOCH:", epoch + 1)
    print("TRAIN")
    train(dataset.train_loader, model, criterion, optimizer, device)
    print("VALIDATION")
    validate(dataset.val_loader, model, criterion, device)
    print("GENERATION")
    generate(model, device, num_images=100)

We can see that initially the samples do not look like the data from our training distribution, but after some epochs of optimizing the generative model we are able to generate images by decoding the samples from our Gaussian prior. 

# Additional exercises

1. Use a 2 dimensional latent space and visualize the latent space of the trained model as well as traverse deterministic z values during generation to see how the model interpolates between different concepts. 
2. Try out last week's pre-training and semi-supervised classification scenarios. Note that as you receive samples z ~ q(z|x) that you now classify p(y|z), you can use multiple samples to get uncertainty estimates! Try to see how this impacts your classification scenario. 